# Trabajo MC
## Redundancy Allocation Problem with Tabu Search

#### TO-DO LIST
- Implementar búsqueda tabú x
 - <span style="color:gray">(Tener en cuenta el tamaño máximo de la lista tabú {1}) </span>
- Estudiar heurística explicada en el paper
- Aplicar la heurística al algoritmo
- Desarrollar funciones de generación de problemas
- Desarrollar funciones de representación gráfica


> {1} Según el artículo, este tamaño máximo sera dinámico (hay que investigar más esto)


---
#### Dudas

- Paso 2 del algoritmo (¿cómo se hace el incremento aleatorio de la dirección?)
- mngli vs mnli
- U-function <span style="color:red;font-size:20px"> <-- ¡¡IMPORTANTE!! </span>


---

#### Comentarios

Un multi-state series-parallel system (MSS) es un sistema que tiene un número de componentes puestos en serie. Cada componente puede tener uno o más elementos del mismo tipo puestos en paralelo para aumentar la redundancia, y por tanto la fiabilidad del sistema, pero aumentando también su coste. El objetivo es minimizar el coste del diseño del sistema cumpliendo un requisito mínimo de fiabilidad.

Una vez seleccionada una version de un elemento para un componente, sólo se puede generar redundancia con elementos de esa versión.

- Solución inicial: Cada componente del sistema contiene sólo UN elemento de la VERSIÓN 1
- Incrementar la dirección de la solución actual aleatoriamente
- Aplicar búsqueda tabú al subespacio comprendido por los vecinos de la solución actual


La disponibilidad del sistema es el score que se va a considerar para evaluar la bondad de una solución, y es todo el rollo ese de las transformadas y nosecuanto que se aplica a las fiabilidades nominales (W) de cada elemento individual: Para cada componente, los rendimientos de los distintos elementos en paralelo se suman a través de la u-transformada esa rara. Para todos los componentes del sistema, el rendimiento es el mínimo de los rendimientos de los componentes, pero también a través de la u-transformada. (Miratelo Alvaro que por más que rasco no lo entiendo)

El espacio de estados -denotado como S- tiene (N - 2s + 1) posibles particiones (desde la mínima dirección posible, 2s; hasta la máxima, N).

A la hora de aplicar TS a un subespacio dado, es necesario extraer una estructura de **vecindario** para el mismo. Ésta estructura se obtiene a partir de todos los posibles movimientos únicos aplicados a la solución actual Y. Es necesario puntualizar que durante esta operación, la dirección (suma de las versiones de cada componente y redundancia de cada uno) de los vecinos se mantiene igual respecto a la de la solución original.

Un **movimiento** consiste en sumar y restar uno al número de versión y al de redundancia de cada componente de la solución, lo cual puede realizarse de tres maneras:

- Cambiando el número de redundancia de los componentes
- Cambiando el número de versión de los componentes
- Cambiando tanto el número de redundancia como el número de versión de los componentes

De entre todos los vecinos será necesario **descartar** los que sean tabú en el momento determinado y escoger aquel considerado como **mejor solución (Y’)**. Ésta solución deberá añadirse a la **lista tabú** (una lista limitada donde se almacenan las soluciones escogidas más recientes). <span style='color:#f1c232;'>El tamaño de ésta lista tabú tenemos que ver cómo lo calculamos.</span>  
Además, permitiremos que se pueda seleccionar una solución con una disponibilidad menor a la mínima requerida (A<sub>0</sub>), de tal manera que podamos atravesar la región no factible de soluciones. Para ello **añadiremos una penalización ponderada** a la función de coste para aquellas soluciones que violen las restricciones de disponibilidad.

Criterio de parada:

In [1]:
from TabuSearch import TabuSearch

In [2]:
def str_to_problem(s, structure):
    lines = s.split("\n")
    
    A,C,W = [],[],[]
    for i,l in list(enumerate(lines)):
        if(len(l.strip().split(" "))>4):
            lines[i]=l.strip().split(" ")[1:]
        else:
            lines[i]=l.strip().split(" ")
    c0=0
    for n,c in list(enumerate(structure)):
        A.append([])
        C.append([])
        W.append([])
        for i in range(c):
            A[n].append(float(lines[i+c0][1]))
            C[n].append(float(lines[i+c0][2]))
            W[n].append(float(lines[i+c0][3]))
        c0+=c
    return A,C,W

In [3]:
''' Problem 1 -> lis4-(7/11)-4'''
def problem_1(a_0):
    s = 4
    A_0 = [0.910, 0.920, 0.940, 0.950, 0.960, 0.970, 0.980, 0.990, 0.999] # <- SOLO SELECCIONAR UNO 
    X_max = [10]*4
    J_max = [11,7,9,7]
    
    A, C, W  = str_to_problem("1 1 0.990 1.117 25 \n 2 0.996 1.310 25 \n 3 0.995 1.903 35 \n 4 0.961 1.640 35 \n 5 0.993 2.122 50 \n 6 0.957 1.910 50 \n 7 0.942 1.722 50 \n 8 0.991 2.591 72 \n 9 0.951 2.001 72 \n 10 0.986 3.284 100 \n 11 0.979 3.095 100 \n 2 1 0.967 4.010 40 \n 2 0.914 3.450 40 \n 3 0.960 4.350 55 \n 4 0.953 4.840 78 \n 5 0.920 4.210 78 \n 6 0.950 5.800 93 \n 7 0.948 6.550 110 \n 3 1 0.967 0.636 25 \n 2 0.952 0.448 35 \n 3 0.973 0.868 35 \n 4 0.972 0.964 50 \n 5 0.949 0.678 50 \n 6 0.988 1.096 50 \n 7 0.966 1.358 72 \n 8 0.954 1.298 72 \n 9 0.945 1.810 100 \n 4 1 0.987 0.614 12.5 \n 2 0.985 0.883 25 \n 3 0.961 0.745 25 \n 4 0.980 0.963 30 \n 5 0.958 0.885 30 \n 6 0.974 1.338 45 \n 7 0.982 1.445 45", J_max)    
    W_T = [(100, 4260),(80, 800), (50, 1200), (20, 2496)]
    q = 100
    mnli = 200
    mngi = 500
    def discount_function(c, x):
        m_1_2 = [(3,5), (3,3), (3,3), (2,6)]
        disc_1_2 = [(0.90,0.80), (0.85,0.85), (0.95, 0.95), (0.95, 0.90)]
        if m_1_2[c][0] >= x:
            return 1
        if m_1_2[c][0] < x <= m_1_2[c][1]:
            return disc_1_2[c][0]
        else:
            return disc_1_2[c][1]
        
    return TabuSearch(s, A_0[a_0], X_max, J_max, A, C, W, W_T, q, mnli, mngi,discount_function )

In [4]:
#Si da
p_1 = problem_1(0)
p_1.ts()

[1, 1, 1, 1, 1, 1, 1, 2] 9
[1, 1, 2, 1, 1, 1, 1, 2] 10
[1, 1, 2, 1, 1, 1, 2, 2] 11


[1, 1, 2, 1, 1, 1, 2, 2]

In [5]:
#No da
p_1.score([1, 1, 2, 1, 1, 1, 2, 2])

71.087470674691872

In [8]:
''' Problem 2 -> lev5-(4/9)-4 '''
def problem_2(a_0):
    s = 5

    A_0 = [0.975, 0.98, 0.990] # <- SOLO SELECCIONAR UNO 
    X_max = [10]*4
    J_max = [7,5,4,9,4]
    
    A, C, W  = str_to_problem("1 1 0.980 0.590 120 \n 2 0.977 0.535 100 \n 3 0.982 0.470 85 \n 4 0.978 0.420 85 \n 5 0.983 0.400 48 \n 6 0.920 0.180 31 \n 7 0.984 0.220 26 \n 2 1 0.995 0.205 100 \n 2 0.996 0.189 92 \n 3 0.997 0.091 53 \n 4 0.997 0.056 28 \n 5 0.998 0.042 21 \n 3 1 0.971 7.525 100 \n 2 0.973 4.720 60 \n 3 0.971 3.590 40 \n 4 0.976 2.420 20 \n 4 1 0.977 0.180 115 \n 2 0.978 0.160 100 \n 3 0.978 0.150 91 \n 4 0.983 0.121 72 \n 5 0.981 0.102 72 \n 6 0.971 0.096 72 \n 7 0.983 0.071 55 \n 8 0.982 0.049 25 \n 9 0.977 0.044 25 \n 5 1 0.984 0.986 128 \n 2 0.983 0.825 100 \n 3 0.987 0.490 60 \n 4 0.981 0.475 51",J_max)    
    W_T = [(100, 4203),(80, 788), (50, 1228), (20, 2536)]
    q = 5000
    mnli = 500
    mngi = 4000
    
    return TabuSearch(s, A_0[a_0], X_max, J_max, A, C, W, W_T, q, mnli, mngi)

In [10]:
# Works like a charm <3
p_2 = problem_2(1)
p_2.score([10,10,10,10,10,2,5,2,7,2])

61.93

In [309]:
''' Problem 3 -> lev4-(4/6)-3 '''
def problem_3(a_0):
    s = 4
    A_0 = [0.9, 0.96, 0.99] # <- SOLO SELECCIONAR UNO 
    X_max = [10]*4
    J_max = [5, 4, 6, 5]
    A = [[0.970, 0.964, 0.980, 0.9869, 0.960],
         [0.967, 0.914, 0.960, 0.953],
         [0.959,0.970,0.959, 0.960, 0.970, 0.960],
         [0.989, 0.979, 0.980, 0.960, 0.980]]
    C = [[0.520, 0.620, 0.720, 0.890, 1.020],
         [0.516, 0.916, 0.967, 1.367],
         [0.214, 0.384, 0.534, 0.614, 0.783, 0.813],
         [0.683, 0.645, 0.697, 1.190, 1.260]]
    W = [[50, 80, 80, 100, 150],
         [20, 50, 50, 75],
         [60, 90, 180, 200, 200, 240],
         [25, 25, 30, 70, 70]]
    W_T = [(100, 20),(80, 30), (40, 50)]
    q = 100
    mnli = 200
    mngi = 500
    
    return TabuSearch(s, A_0[a_0], X_max, J_max, A, C, W, W_T, q, mnli, mngi)

In [310]:
p_3 = problem_3(0)
p_3.score([1,2,3,2,4,3,1,5])

5.986

In [14]:
''' Problem 4 '''
def problem_4(a_0):
    s = 6
    A_0 = [0.975, 0.98, 0.99] # <- SOLO SELECCIONAR UNO 
    X_max = [10]*6
    J_max = [11, 8, 9, 4, 4, 6]
    
    A, C, W  = str_to_problem("1 1 0.932 1.590 27.3 \n 2 0.998 0.515 27.7 \n 3 0.983 0.225 49.8 \n 4 0.927 3.220 52.5 \n 5 0.959 4.020 62.0 \n 6 0.955 4.270 66.4 \n 7 0.984 3.670 84.6 \n 8 0.918 4.630 90.7 \n 9 0.939 1.010 97.0 \n 10 0.988 0.779 124 \n 11 0.984 3.130 129 \n 2 1 0.989 0.050 35.9 \n 2 0.923 1.290 44.7 \n 3 0.900 0.204 51.4 \n 4 0.946 2.220 63.2 \n 5 0.917 0.872 68.8 \n 6 0.962 1.830 81.8 \n 7 0.994 0.294 82.0 \n 8 0.984 2.810 115 \n 3 1 0.931 3.620 34.7 \n 2 0.950 0.475 41.0 \n 3 0.911 1.170 41.4 \n 4 0.956 0.793 43.6 \n 5 0.966 3.740 48.6 \n 6 0.992 4.590 59.6 \n 7 0.929 1.740 66.2 \n 8 0.968 1.720 91.9 \n 9 0.901 1.300 121 \n 4 1 0.915 2.490 25.1 \n 2 0.908 0.078 28.8 \n 3 0.928 1.370 50.2 \n 4 0.944 4.470 129 \n 5 1 0.908 1.550 34.9 \n 2 0.980 4.920 64.3 \n 3 0.964 2.650 108 \n 4 0.924 4.720 126 \n 6 1 0.965 3.220 24.8 \n 2 0.927 2.890 47.3 \n 3 0.986 3.410 58.8 \n 4 0.983 1.920 107 \n 5 0.991 4.510 120 \n 6 0.954 4.580 125", J_max)    
    W_T = [(100, 4203),(80, 788), (50, 1228), (20, 2536)]
    q = 400
    mnli = 35
    mngi = 100
    
    return TabuSearch(s, A_0[a_0], X_max, J_max, A, C, W, W_T, q, mnli, mngi)

In [312]:
p_4 = problem_4(0)
print(p_)
p_4.score([4,4,5,7,2,1,3,1,2,2,3,4])

NameError: name 'problem_4' is not defined

In [36]:
print(p_1.X_max+p_1.J_max)
p_1.random_modification([10,10,10,10,11,7,9,6])

[10, 10, 10, 10, 11, 7, 9, 7]
1 [10, 11, 10, 10, 11, 7, 9, 6] [0, 2, 3, 4, 5, 6, 7]
4 [10, 10, 10, 10, 12, 7, 9, 6] [0, 2, 3, 5, 6, 7]
2 [10, 10, 11, 10, 11, 7, 9, 6] [0, 3, 5, 6, 7]
5 [10, 10, 10, 10, 11, 8, 9, 6] [0, 3, 6, 7]
0 [11, 10, 10, 10, 11, 7, 9, 6] [3, 6, 7]
6 [10, 10, 10, 10, 11, 7, 10, 6] [3, 7]
3 [10, 10, 10, 11, 11, 7, 9, 6] [7]
7 [10, 10, 10, 10, 11, 7, 9, 7] []


[10, 10, 10, 10, 11, 7, 9, 7]

In [37]:
random.choice([1,4,5,7])

5

In [11]:
import numpy as np
from scipy.stats import binom

In [12]:
solution = [8, 8, 8, 9, 5, 4, 6, 5]
J_max = [11,7,9,7]
W_T = [(100, 4260),(80, 800), (50, 1200), (20, 2496)]
A, C, W  = str_to_problem("1 1 0.990 1.117 25 \n 2 0.996 1.310 25 \n 3 0.995 1.903 35 \n 4 0.961 1.640 35 \n 5 0.993 2.122 50 \n 6 0.957 1.910 50 \n 7 0.942 1.722 50 \n 8 0.991 2.591 72 \n 9 0.951 2.001 72 \n 10 0.986 3.284 100 \n 11 0.979 3.095 100 \n 2 1 0.967 4.010 40 \n 2 0.914 3.450 40 \n 3 0.960 4.350 55 \n 4 0.953 4.840 78 \n 5 0.920 4.210 78 \n 6 0.950 5.800 93 \n 7 0.948 6.550 110 \n 3 1 0.967 0.636 25 \n 2 0.952 0.448 35 \n 3 0.973 0.868 35 \n 4 0.972 0.964 50 \n 5 0.949 0.678 50 \n 6 0.988 1.096 50 \n 7 0.966 1.358 72 \n 8 0.954 1.298 72 \n 9 0.945 1.810 100 \n 4 1 0.987 0.614 12.5 \n 2 0.985 0.883 25 \n 3 0.961 0.745 25 \n 4 0.980 0.963 30 \n 5 0.958 0.885 30 \n 6 0.974 1.338 45 \n 7 0.982 1.445 45", J_max)    

## 

In [13]:
def rec(index):
    if(index == 3):                   
        return list(range(1, solution[index]+1))
    else:
        return [ [x, c] if( type(c) is not list) else [x]+c for x in range(1, solution[index]+1) for c in rec(index+1) ]
def extract_possible_m():
    return rec(0)

In [14]:
%time len(rec(0))

CPU times: user 4.86 ms, sys: 7 µs, total: 4.87 ms
Wall time: 4.75 ms


4608

In [15]:
rec=rec(0)

In [16]:
%time len(sorted(rec,key=lambda k: m_performance(k)))

NameError: name 'm_performance' is not defined

In [17]:
%time m_performance([8,8,8,9])

NameError: name 'm_performance' is not defined

In [18]:
def m_performance(k_list):
    return min(k_list[i]*ij_performance(solution[4:][i],i) for i in range(len(k_list)))

In [19]:
def ij_performance(version,component):            
            return W[component][version-1] 

In [159]:
def delta(m):
    alpha = lambda k,x_i,a_ij: binom.pmf(k,x_i,a_ij)
    #return np.prod([alpha(m[c],solution[:4][c],A[c][solution[4:][c]-1]) for c in range(len(m))])
    return np.prod([binom.cdf(m[c],solution[c],A[c][solution[4+c]-1]) for (c,_) in enumerate(m)])

In [190]:
%time delta([10]*4)

CPU times: user 699 µs, sys: 7 µs, total: 706 µs
Wall time: 627 µs


1.0

In [186]:
%time binom.pmf(9,9,A[0][5])

CPU times: user 402 µs, sys: 0 ns, total: 402 µs
Wall time: 347 µs


0.67329817917589907

In [316]:
def cost():  
    discount_func = lambda c,x:1
    return sum(discount_func(c, solution[c])*solution[c] * C[c][j-1] for c, j in enumerate(solution[4:]))

In [317]:
%time cost()

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 38.4 µs


77.357

In [324]:
def availability():
    possible_m = extract_possible_m()
    return ( (1/sum([ t_k for w_k, t_k in W_T ]))
        *sum([
            delta_summ(k, possible_m)
            for k, _ in enumerate(W_T)
        ]) )

def delta_summ(k_interval,k_lists):
    m_sort= sorted(k_lists,key=lambda k: m_performance(k))
    return W_T[k_interval][1]*sum(delta(m) for m in m_sort if m_performance(m)>=W_T[k_interval][0])

In [331]:
%time delta_summ(0,recero)

CPU times: user 1.05 s, sys: 53.2 ms, total: 1.1 s
Wall time: 951 ms


4259.9982236821834

In [435]:
recero = extract_possible_m()
m_sort = list(filter(lambda k: m_performance(k)>=W_T[0][0],recero))
len(m_sort)

2058

In [436]:
%time len(sorted(recero,key=lambda k: m_performance(k)))

CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 14 ms


4608

In [438]:
%time W_T[0][1]*sum(delta(m) for m in m_sort)

CPU times: user 995 ms, sys: 53.2 ms, total: 1.05 s
Wall time: 918 ms


4259.9982236821834